In [23]:
import numpy as np
import scipy.integrate as integrate
from scipy import linalg
import matplotlib.pyplot as plt

%matplotlib inline


In [24]:
def weight(x):
    '''
    Define weight function
    '''
    ans = np.sin(np.log(x + 1))
    return ans

def integrand(x, i, j):
    '''
    Define integrand for the entries
    of the correlation matrix
    '''
    p = i + j
    ans = x**p*weight(x)
    return ans

def M_elem(i, j):
    '''
    Define the (i,j) element of the 
    correlation matrix M
    '''
    ans = integrate.quad(integrand, 0, 1, args = (i, j))[0]
    return ans

In [25]:
def create_M(ord):
    '''
    Create correlation matrix M 
    '''
    ans = np.array([M_elem(i, j) for i in range(ord + 1) for j in range(ord + 1)]).reshape(ord + 1, ord + 1)
    return ans

In [26]:
ord_pol = 5
M = create_M(ord_pol)
print M

[[0.36972237 0.23722455 0.17394447 0.13710581 0.1130647  0.09616204]
 [0.23722455 0.17394447 0.13710581 0.1130647  0.09616204 0.0836387 ]
 [0.17394447 0.13710581 0.1130647  0.09616204 0.0836387  0.07399217]
 [0.13710581 0.1130647  0.09616204 0.0836387  0.07399217 0.06633541]
 [0.1130647  0.09616204 0.0836387  0.07399217 0.06633541 0.06011148]
 [0.09616204 0.0836387  0.07399217 0.06633541 0.06011148 0.05495326]]


In [27]:
# Until this point all seems fine

In [28]:
# Now we use Cholesky decomp. 
L = linalg.cholesky(M)
L_inv = linalg.inv(L)
print L_inv

[[ 1.64460699e+00 -4.35221622e+00  7.78361074e+00 -1.18172980e+01
   1.63748194e+01 -2.14019465e+01]
 [ 0.00000000e+00  6.78307428e+00 -3.23507993e+01  9.17517877e+01
  -2.02659391e+02  3.85031445e+02]
 [ 0.00000000e+00  0.00000000e+00  2.75756308e+01 -1.87731150e+02
   7.24942769e+02 -2.09607563e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.11287328e+02
  -9.82376045e+02  4.79138903e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   4.47631499e+02 -4.85198631e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  1.79733849e+03]]


In [48]:
def pols(x):
    '''
    Create array of polynomials whose 
    coefficients are given by the elements of
    L_inv
    '''
    x_vec = np.array([x**i for i in range(6)])
    ans = L_inv.T.dot(x_vec)
    return ans

# Here I choose any two polynomials of the array (0 and 1, in this case) and 
# check orthogonality with the weight. It is not working. 
def check_pol_ort(x):
    ans = pols(x)[1]*pols(x)[1]*weight(x)
    return ans

integrate.quad(check_pol_ort, 0, 1)[0]

1.0000000000000018

In [41]:
arr = np.array([[1],[2],[3],[4]])
print arr
print arr.T

[[1]
 [2]
 [3]
 [4]]
[[1 2 3 4]]


In [30]:
dom = np.linspace(-100, 100, 1e3)
plt.plot(dom, pol(dom)[0])
plt.plot(dom, test_pol(dom), 'r--',)

/Users/Manu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  if __name__ == '__main__':


NameError: name 'pol' is not defined

In [ ]:
def test_pol(x):
    ans = 1.64 - 4.35*x + 7.78*x**2 - 11.8*x**3 + 16.3*x**4 - 21.4*x**5
    return ans



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate

def intmono(n,m):
    '''
    Calculate the integral of \int_0^1 x^n x^m sin(ln(x+10)) dx
    Method is straight sum (not optimal but fast)
    

    Parameters
    ----------
    n: int
        exponent of x (first)
    m: int
        exponent of x (second)
        
    Returns
    ----------
    float
        result of integration
        
    '''
    
#   Number of points to perform the integration over
    points = 10000
    x = np.linspace(0,1,points)
#   Create integrand
    wgt = np.sin(np.log(x+1))
    p1 = x**n
    p2 = x**m
    integrand = wgt*p1*p2/points
    
#   Integral is just sum
    return sum(integrand)

def intpoly(L,n,m):
    '''
    Produce the integral of two polynomials from 0 to 1 with a 
    specific weight function:
    
    \int_0^1 Sin(Ln(x+1)) P_n(x) P_m(x)    

    Parameters
    ----------
    L: array, float
        Coefficents for set of polynomials
    n: int
        order of first polynomial
    m: int
        order of second polynomial
    Returns
    ----------
    float
        result of integration
    
    '''
    
#   Number of points to perform the integration over
    points = 10000
    x = np.linspace(0,1,points)
    
#   Create integrand using polynomial generator from numpy
    wgt = np.sin(np.log(x+1))
    p1 = np.poly1d(L[n,::-1])
    p2 = np.poly1d(L[m,::-1])
    integrand = wgt*p1(x)*p2(x)/points
    
#   Integral is just sum
    return sum(integrand)

# Vectorise just converts a scalar function to one that accepts vectors
# Needed for numpy.fromfunction
tmp = np.vectorize(intmono)

# Creates an array with entries calculated from the function which takes array indexes as input
M = np.fromfunction(tmp,(6,6))

# Cholesky decomposition. This is equivalent to Gram-Schmidtt
# orthonormalisation but more stable
L = np.linalg.cholesky(M)

# Calculate inverse
L = np.linalg.inv(L)

# Check polynomials are now orthonormal
I = np.zeros((6,6), dtype='float') 
for i in range(6):
    for j in range(6):
        I[i,j] = intpoly(L,i,j)

# This set global precision level for print statements    
np.set_printoptions(precision=3)
print(M)
print(I)

In [ ]:
# Exercise 2

import matplotlib.pyplot as plt
% matplotlib inline

ang = np.linspace(0, np.pi)

plt.plot(np.cos(ang), np.sin(ang), linewidth = 10, color = 'b')
plt.plot(0.96*np.cos(ang), 0.96*np.sin(ang), linewidth = 10, color = 'g')
plt.plot(0.92*np.cos(ang), 0.92*np.sin(ang), linewidth = 10, color = 'r')
plt.plot(0.88*np.cos(ang), 0.88*np.sin(ang), linewidth = 10, color = 'c')
plt.plot(0.84*np.cos(ang), 0.84*np.sin(ang), linewidth = 10, color = 'm')
plt.plot(0.80*np.cos(ang), 0.80*np.sin(ang), linewidth = 10, color = 'y')
plt.plot(0.76*np.cos(ang), 0.76*np.sin(ang), linewidth = 10, color = 'k')